# COVID19 - US - Maps

Ana Cláudia Ramos Martins Abreu A80276

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from qgis.core import *
from qgis.PyQt.QtGui import *
from qgis.PyQt.QtCore import *
from IPython.display import Image

In [2]:
QgsApplication.setPrefixPath(r'C:\\OSGeo4W64\\apps\\qgis', True)
qgs = QgsApplication([], True)
qgs.initQgis()
sys.path.append(r'C:\OSGeo4W64\apps\qgis\python\plugins')
project = QgsProject.instance()

Load map layer

In [3]:
path = "C:\\Users\\Claudia\\Desktop\\Trabalho SIG\\Mapa US2\\s_11au16.shp"
map_layer = QgsVectorLayer(path, 'USA map', 'ogr')

if not map_layer.isValid():
    print("Failed to load the layer!")
else:
    project.addMapLayer(map_layer)
    print("Sucess")

Sucess


Load csv layer

In [4]:
csv_path = "file:///C:/OSGeo4W64/bin/QGIS/us_covid19.csv?delimiter=,'"
csv_layer = QgsVectorLayer(csv_path, 'Data', 'delimitedtext')

if not csv_layer.isValid():
    print('Layer failed to load!')
else:
    project.addMapLayer(csv_layer)
    print("Sucess")

Sucess


JOIN map layer with csv layer

In [5]:
joinName = 'state'
targetName = 'STATE'

joinObject = QgsVectorLayerJoinInfo()
joinObject.setJoinFieldName(joinName)
joinObject.setTargetFieldName(targetName)
joinObject.setJoinLayerId(csv_layer.id())
joinObject.setUsingMemoryCache(True)
joinObject.setJoinLayer(csv_layer)
flag = map_layer.addJoin(joinObject)

### Distribuição dos casos positivos

Criar uma nova camada para imprimir os casos confirmados

In [6]:
positive_layer = map_layer.clone()
positive_layer.setName("Casos Confirmados")

In [7]:
from qgis.PyQt import QtGui

myOpacity = 1

myTargetField = 'Data_positive'
myRangeList = []

myMin = 0
myMax = 0
myLabel = '0'
myColour = QtGui.QColor('#e6e6ff')
mySymbol = QgsSymbol.defaultSymbol(positive_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 0
myMax = 1000
myLabel = '0-1000'
myColour = QtGui.QColor('#ff6666')
mySymbol = QgsSymbol.defaultSymbol(positive_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 1000
myMax = 10000
myLabel = '1000-10000'
myColour = QtGui.QColor('#ff3333')
mySymbol = QgsSymbol.defaultSymbol(positive_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 10000
myMax = 100000
myLabel = '10000-100000'
myColour = QtGui.QColor('#cc0000')
mySymbol = QgsSymbol.defaultSymbol(positive_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 100000
myMax = 500000
myLabel = '100000-500000'
myColour = QtGui.QColor('#990000')
mySymbol = QgsSymbol.defaultSymbol(positive_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myMin = 500000
myMax = 1000000
myLabel = '500000-1000000'
myColour = QtGui.QColor('#000000')
mySymbol = QgsSymbol.defaultSymbol(positive_layer.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

myRenderer = QgsGraduatedSymbolRenderer('', myRangeList)
myClassificationMethod = QgsApplication.classificationMethodRegistry().method("EqualInterval")
myRenderer.setClassificationMethod(myClassificationMethod)
myRenderer.setClassAttribute(myTargetField)

positive_layer.setRenderer(myRenderer)

In [8]:
xt = map_layer.extent()
# print(xt)
width  = 2000
height = int(width*xt.height()/xt.width())
print("Gerar mapa com {} por {}".format(width, height))


project = QgsProject().instance()
project.addMapLayers([positive_layer])
manager = project.layoutManager()
layoutName = 'positive'
layouts_list = manager.printLayouts()
for layout in layouts_list:
    if layout.name() == layoutName:
        manager.removeLayout(layout)
        
layout = QgsPrintLayout(project)
layout.initializeDefaults()
layout.setName(layoutName)
manager.addLayout(layout)

pc = layout.pageCollection()
pc.page(0).attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))


map = QgsLayoutItemMap(layout)
map.setRect(20,20,20,20)
map.setLayers([positive_layer])



ms = QgsMapSettings()
ms.setLayers([positive_layer])
ms.setExtent(xt)
map.setExtent(xt)


layout.addLayoutItem(map)



map.attemptMove(QgsLayoutPoint(0,0,QgsUnitTypes.LayoutMillimeters))
map.attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))


map.setBackgroundColor(QColor(250,250,250))




legend = QgsLayoutItemLegend(layout)

legend.setReferencePoint(6)
legend.setScale(3)
legend.attemptMove(QgsLayoutPoint(0,height,QgsUnitTypes.LayoutMillimeters))

layout.addLayoutItem(legend)
legend.setLegendFilterByMapEnabled(True)

legend.setLinkedMap(map)

Gerar mapa com 2000 por 260


In [9]:
exporter = QgsLayoutExporter(manager.layoutByName("positive"))

#Se fizer render apenas uma vez o reference point continua a ser top left e não bottom left
image = exporter.renderPageToImage(0)
image = exporter.renderPageToImage(0)

from IPython.display import Image
imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image.save( imgbuf,"PNG" )
Image( imgbuf.data() )